#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.
    * 성능 가이드
            * Accuracy : 0.980~1.00


## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 다음의 두가지 방법 중 하나를 선택하여 폴더를 준비하고 데이터를 로딩하시오.
        * 1) 로컬 수행(Ananconda)
            * 제공된 압축파일을 다운받아 압축을 풀고
            * anaconda의 root directory(보통 C:/Users/< ID > 에 project 폴더를 만들고, 복사해 넣습니다.
        * 2) 구글콜랩
            * 구글 드라이브 바로 밑에 project 폴더를 만들고,
            * 데이터 파일을 복사해 넣습니다.
    
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### (1) 경로 설정

#### 1) 로컬 수행(Anaconda)
* project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
path = 'C:/Users/User/Desktop/'

#### 2) 구글 콜랩 수행

* 구글 드라이브 연결

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/project/'

### (2) 라이브러리 불러오기

#### 1) 라이브러리 로딩

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

#### 2) 제공 함수 생성
* 변수 중요도를 시각화할 수 있는 함수를 제공합니다.
* 입력 :
    * importance : 트리모델의 변수 중요도(예: model.feature_importances_)
    * names : 변수 이름 목록(예 : x_train.columns
    * result_only  : 변수 중요도 순으로 데이터프레임만 return할지, 그래프도 포함할지 결정. False이면 결과 데이터프레임 + 그래프
    * topn : 중요도 상위 n개만 표시. all 이면 전체.
* 출력 :
    * 중요도 그래프 : 중요도 내림차순으로 정렬
    * 중요도 데이터프레임 : 중요도 내림차순으로 정렬

In [ ]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (3) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
    * data01_test.csv : 테스트용
    * feature.csv : feature 이름을 계층구조로 정리한 데이터

* 세부 요구사항
    * 칼럼 삭제 : data01_train.csv와 data01_test.csv 에서 'subject' 칼럼은 불필요하므로 삭제합니다.

#### 1) 데이터로딩

In [16]:
# train, test 데이터 불러오기
data_train = pd.read_csv("data01_train.csv")
data_train.drop('subject', axis=1, inplace=True)
data_train['Activity'] = data_train['Activity'].astype('category')

data_test = pd.read_csv('data01_test.csv')
data_test.drop('subject', axis=1, inplace=True)
data_test['Activity'] = data_test['Activity'].astype('category')

In [5]:
features = pd.read_csv('features.csv')

In [6]:
feature_imp = pd.read_csv('feature_importances.csv')

#### 2) 기본 정보 조회

In [17]:
# train 데이터 기본정보
display(data_train.head())
print(data_train.info())
display(data_train.describe())

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: category(1), float64(561)
memory usage: 25.2 MB
None


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


In [18]:
# test 데이터 기본정보
display(data_test.head())
print(data_test.info())
display(data_test.describe())

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.284379,-0.021981,-0.116683,-0.992490,-0.979640,-0.963321,-0.992563,-0.977304,-0.958142,-0.938850,...,-0.509523,-0.850065,-0.018043,0.092304,0.074220,-0.714534,-0.671943,-0.018351,-0.185733,SITTING
1,0.277440,-0.028086,-0.118412,-0.996620,-0.927676,-0.972294,-0.997346,-0.931405,-0.971788,-0.939837,...,-0.210792,-0.613367,-0.022456,-0.155414,0.247498,-0.112257,-0.826816,0.184489,-0.068699,STANDING
2,0.305833,-0.041023,-0.087303,0.006880,0.182800,-0.237984,0.005642,0.028616,-0.236474,0.016311,...,0.579587,0.394388,-0.362616,0.171069,0.576349,-0.688314,-0.743234,0.272186,0.053101,WALKING
3,0.276053,-0.016487,-0.108381,-0.995379,-0.983978,-0.975854,-0.995877,-0.985280,-0.974907,-0.941425,...,-0.566291,-0.841455,0.289548,0.079801,-0.020033,0.291898,-0.639435,-0.111998,-0.123298,SITTING
4,0.271998,0.016904,-0.078856,-0.973468,-0.702462,-0.869450,-0.979810,-0.711601,-0.856807,-0.920760,...,0.447577,0.214219,0.010111,0.114179,-0.830776,-0.325098,-0.840817,0.116237,-0.096615,STANDING


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: category(1), float64(561)
memory usage: 6.3 MB
None


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,...,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000,1471.000000
mean,0.273198,-0.017281,-0.108123,-0.614634,-0.515427,-0.607533,-0.639950,-0.530760,-0.609256,-0.481055,...,0.118651,-0.311513,-0.632273,0.009324,0.015666,0.006272,-0.001510,-0.481737,0.055771,-0.064194
std,0.079989,0.045957,0.049082,0.448480,0.506094,0.424243,0.422994,0.489381,0.418536,0.542756,...,0.258112,0.313792,0.296179,0.324864,0.452616,0.608954,0.483028,0.522714,0.298124,0.281645
min,-1.000000,-1.000000,-0.418354,-0.999717,-0.999873,-1.000000,-0.999867,-1.000000,-0.999879,-0.948723,...,-1.000000,-0.995357,-0.994664,-0.937468,-0.990492,-0.995222,-0.969066,-0.999380,-0.995073,-1.000000
25%,0.263787,-0.024792,-0.120733,-0.992669,-0.979082,-0.981070,-0.993498,-0.979214,-0.980659,-0.936791,...,-0.035193,-0.546342,-0.844547,-0.119166,-0.265533,-0.485998,-0.380300,-0.814060,-0.017413,-0.148445
50%,0.277322,-0.017187,-0.108124,-0.952426,-0.867309,-0.869890,-0.958705,-0.873891,-0.863451,-0.890491,...,0.135282,-0.347433,-0.706699,0.005049,0.023421,-0.005036,0.002408,-0.708911,0.178814,-0.002243
75%,0.288058,-0.010238,-0.096606,-0.245405,-0.030639,-0.260223,-0.297260,-0.058824,-0.256657,-0.030692,...,0.289832,-0.125796,-0.516910,0.135698,0.312690,0.518184,0.374583,-0.486534,0.248126,0.096674
max,0.631510,0.359587,0.543939,0.899922,0.782590,0.931308,0.950758,0.602458,0.784041,0.821218,...,0.860512,0.941113,0.894210,0.980889,0.991899,0.994366,0.979522,1.000000,0.432496,0.992766


In [11]:
display(feature_imp.head())
print(feature_imp.info())
display(feature_imp.describe())

,feature_names,fi_dynamic,fi_standing,fi_sitting,fi_laying,fi_walking,fi_walkingup,fi_walkingdown
0,tBodyAcc-mean()-X,0.000018,0.000223,0.000296,0.000029,0.000106,0.000287,0.000117
1,tBodyAcc-mean()-Y,0.000009,0.000363,0.000567,0.000145,0.000198,0.000310,0.000469
2,tBodyAcc-mean()-Z,0.000000,0.000263,0.000413,0.000149,0.000185,0.000328,0.000098
3,tBodyAcc-std()-X,0.000038,0.006221,0.000707,0.000178,0.011663,0.006478,0.029259
4,tBodyAcc-std()-Y,0.000000,0.001981,0.000216,0.000384,0.000322,0.000418,0.000923


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   feature_names   561 non-null    object 
 1   fi_dynamic      561 non-null    float64
 2   fi_standing     561 non-null    float64
 3   fi_sitting      561 non-null    float64
 4   fi_laying       561 non-null    float64
 5   fi_walking      561 non-null    float64
 6   fi_walkingup    561 non-null    float64
 7   fi_walkingdown  561 non-null    float64
dtypes: float64(7), object(1)
memory usage: 35.2+ KB
None


,fi_dynamic,fi_standing,fi_sitting,fi_laying,fi_walking,fi_walkingup,fi_walkingdown
count,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000
mean,0.001783,0.001783,0.001783,0.001783,0.001783,0.001783,0.001783
std,0.007283,0.005449,0.004702,0.009749,0.003740,0.003900,0.005389
min,0.000000,0.000000,0.000018,0.000000,0.000013,0.000012,0.000013
25%,0.000000,0.000266,0.000315,0.000016,0.000149,0.000174,0.000191
50%,0.000000,0.000402,0.000524,0.000058,0.000313,0.000426,0.000354
75%,0.000000,0.000902,0.001075,0.000227,0.001014,0.001595,0.000775
max,0.069982,0.056396,0.050450,0.116279,0.027485,0.036041,0.045597


## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [19]:
data_train['Activity_dynamic'] = data_train['Activity'].replace({'STANDING': 0,
                                                                 'SITTING': 0,
                                                                 'LAYING': 0,
                                                                 'WALKING': 1,
                                                                 'WALKING_UPSTAIRS': 1,
                                                                 'WALKING_DOWNSTAIRS': 1})
print(data_train['Activity'].value_counts())
print(data_train['Activity_dynamic'].value_counts())

Activity
LAYING                1115
STANDING              1087
SITTING               1032
WALKING                998
WALKING_UPSTAIRS       858
WALKING_DOWNSTAIRS     791
Name: count, dtype: int64
Activity_dynamic
0    3234
1    2647
Name: count, dtype: int64


In [75]:
x = data_train.drop(['Activity', 'Activity_dynamic'], axis=1)
y1 = data_train['Activity']
y2 = data_train['Activity_dynamic']

from sklearn.model_selection import train_test_split
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x, y1, y2, test_size=0.3, random_state=1)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) Logistic Regression

In [76]:
model_lg = LogisticRegression(random_state=1)
model_lg.fit(x_train, y2_train)

LogisticRegression(random_state=1)

In [77]:
y_pred_lg = model_lg.predict(x_val)
print(classification_report(y2_val, y_pred_lg))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       954
           1       1.00      1.00      1.00       811

    accuracy                           1.00      1765
   macro avg       1.00      1.00      1.00      1765
weighted avg       1.00      1.00      1.00      1765



#### 2) RandomForest

In [78]:
model_rf = RandomForestClassifier(random_state=1)
model_rf.fit(x_train, y2_train)

RandomForestClassifier(random_state=1)

In [79]:
y_pred_rf = model_lg.predict(x_val)
print(classification_report(y2_val, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       954
           1       1.00      1.00      1.00       811

    accuracy                           1.00      1765
   macro avg       1.00      1.00      1.00      1765
weighted avg       1.00      1.00      1.00      1765



### (2) 단계2-1 : 정적 동작 세부 분류

In [60]:
y_pred_rf.shape

(1765,)

In [62]:
x_val.shape

(1765, 561)

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [34]:
df_stop = data_train[data_train['Activity'].isin(['LAYING', 'SITTING', 'STANDING'])]
df_stop['Activity'].value_counts()

Activity
LAYING                1115
STANDING              1087
SITTING               1032
WALKING                  0
WALKING_DOWNSTAIRS       0
WALKING_UPSTAIRS         0
Name: count, dtype: int64

In [35]:
df_stop.shape

(3234, 563)

In [36]:
df_stop.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_dynamic
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING,0
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING,0
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING,0
7,0.272026,-0.001329,-0.125491,-0.992068,-0.912985,-0.972451,-0.994752,-0.943141,-0.976428,-0.925446,...,-0.704995,-0.024442,0.076332,0.741277,0.729812,-0.817201,0.037746,0.136129,STANDING,0
8,0.284338,0.021956,-0.006925,-0.980153,-0.838394,-0.782357,-0.983683,-0.816199,-0.743923,-0.914011,...,-0.400197,0.021212,-0.009465,-0.282762,0.563343,-0.782072,0.242834,-0.025285,STANDING,0


In [37]:
x_stop = df_stop.drop(['Activity', 'Activity_dynamic'], axis=1)
y_stop = df_stop['Activity']

x_train, x_val, y_train, y_val = train_test_split(x_stop, y_stop, test_size=0.3, random_state=1)

- 랜덤포레스트 모델 사용

In [38]:
model_rf = RandomForestClassifier(random_state=1)
model_rf.fit(x_train, y_train)

RandomForestClassifier(random_state=1)

In [40]:
y_pred_rf = model_rf.predict(x_val)
print(classification_report(y_val, y_pred_rf))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       320
     SITTING       0.94      0.95      0.95       317
    STANDING       0.95      0.95      0.95       334

    accuracy                           0.97       971
   macro avg       0.97      0.97      0.97       971
weighted avg       0.97      0.97      0.97       971



- 로지스틱 회귀 사용

In [41]:
mdoel_lg = LogisticRegression(random_state=1)
model_lg.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [43]:
y_pred_lg = model_lg.predict(x_val)
print(classification_report(y_val, y_pred_lg))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       320
     SITTING       0.95      0.97      0.96       317
    STANDING       0.97      0.96      0.96       334

    accuracy                           0.97       971
   macro avg       0.97      0.97      0.97       971
weighted avg       0.97      0.97      0.97       971



### (3) 단계2-2 : 동적 동작 세부 분류

In [44]:
df_act = data_train[data_train['Activity'].isin(['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'])]
df_act['Activity'].value_counts()

Activity
WALKING               998
WALKING_UPSTAIRS      858
WALKING_DOWNSTAIRS    791
LAYING                  0
SITTING                 0
STANDING                0
Name: count, dtype: int64

* 세부 요구사항
    * 동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [45]:
x_stop = df_act.drop(['Activity', 'Activity_dynamic'], axis=1)
y_stop = df_act['Activity']

x_train, x_val, y_train, y_val = train_test_split(x_stop, y_stop, test_size=0.3, random_state=1)

In [46]:
model_rf = RandomForestClassifier(random_state=1)
model_rf.fit(x_train, y_train)

RandomForestClassifier(random_state=1)

In [47]:
y_pred_rf = model_rf.predict(x_val)
print(classification_report(y_val, y_pred_rf))

                    precision    recall  f1-score   support

           WALKING       0.99      0.99      0.99       307
WALKING_DOWNSTAIRS       0.98      0.97      0.98       243
  WALKING_UPSTAIRS       0.98      0.99      0.98       245

          accuracy                           0.98       795
         macro avg       0.98      0.98      0.98       795
      weighted avg       0.98      0.98      0.98       795



In [48]:
mdoel_lg = LogisticRegression(random_state=1)
model_lg.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [49]:
y_pred_lg = model_lg.predict(x_val)
print(classification_report(y_val, y_pred_lg))

                    precision    recall  f1-score   support

           WALKING       1.00      1.00      1.00       307
WALKING_DOWNSTAIRS       1.00      0.99      0.99       243
  WALKING_UPSTAIRS       0.99      1.00      0.99       245

          accuracy                           0.99       795
         macro avg       0.99      0.99      0.99       795
      weighted avg       0.99      0.99      0.99       795



### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들어서 분류모델 합치기

In [73]:
def model_pipline(data):
    # 동적 구분 칼럼 추가
    data['Activity_dynamic'] = data['Activity'].replace({'STANDING': 0,
                                                         'SITTING': 0,
                                                         'LAYING': 0,
                                                         'WALKING': 1,
                                                         'WALKING_UPSTAIRS': 1,
                                                         'WALKING_DOWNSTAIRS': 1})
    x = data.drop(['Activity', 'Activity_dynamic'], axis=1)
    y1 = data['Activity']
    y2 = data['Activity_dynamic']

    model1 = RandomForestClassifier(random_state=1)
    model1.fit(x, y2)

    y_pred_rf = model1.predict(x)
    data['predict'] = y_pred_rf

    df_stop = data[data['predict'] == 0]
    x_stop = df_stop.drop(['Activity', 'Activity_dynamic'], axis=1)
    y_stop = df_stop['Activity']

    df_act = data[data['predict'] == 1]
    x_act = df_act.drop(['Activity', 'Activity_dynamic'], axis=1)
    y_act = df_act['Activity']

    x_train, x_val, y_train, y_val = train_test_split(x_stop, y_stop, test_size=0.3, random_state=1)
    x_train2, x_val2, y_train2, y_val2 = train_test_split(x_act, y_act, test_size=0.3, random_state=1)

    model2_0 = RandomForestClassifier(random_state=1)
    model2_0.fit(x_train, y_train)
    y_pred_rf2_0 = model2_0.predict(x_val)

    print('정적 동작 예측모델 평가')
    print(classification_report(y_val, y_pred_rf2_0))

    model2_1 = RandomForestClassifier(random_state=1)
    model2_1.fit(x_train2, y_train2)
    y_pred_rf2_1 = model2_1.predict(x_val2)

    print('동적 동작 예측모델 평가')
    print(classification_report(y_val2, y_pred_rf2_1))

#### 2) test 셋으로 예측하고 평가하기

* 성능 평가

In [74]:
model_pipline(data_train)

정적 동작 예측모델 평가
              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       320
     SITTING       0.95      0.96      0.95       317
    STANDING       0.96      0.95      0.96       334

    accuracy                           0.97       971
   macro avg       0.97      0.97      0.97       971
weighted avg       0.97      0.97      0.97       971

동적 동작 예측모델 평가
                    precision    recall  f1-score   support

           WALKING       0.99      0.99      0.99       307
WALKING_DOWNSTAIRS       0.98      0.98      0.98       243
  WALKING_UPSTAIRS       0.98      0.99      0.99       245

          accuracy                           0.98       795
         macro avg       0.98      0.98      0.98       795
      weighted avg       0.98      0.98      0.98       795

